In [263]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from twython import Twython
import json
import time
from twitter import *

In [310]:
# Get keys from TWITTER_CREDS_FN or https://developer.twitter.com/en/apps/16087482
# NOTE: do not run this cell unless you get the API keys from twitter.com.
CONSUMER_KEY = 'e0cR4oCRsfqMQOIgSeZu18xGm'
CONSUMER_SECRET_KEY = 'ucPIEBnGCk6YJUmM8yV3FYbx8Fta1kzDzo4woPHZu1RdqmwGNp'
ACCESS_TOKEN_SECRET = '9waRhmgfqIUToRbrsqXw2qltAC7oDwgJef3aJV91PToaU'
TWITTER_CREDS_FN = '/Users/Che10/Downloads/.twitter-credentials.json'

if (CONSUMER_KEY != ''):
    twitter = Twython(CONSUMER_KEY, CONSUMER_SECRET_KEY, oauth_version=2)
    ACCESS_TOKEN = twitter.obtain_access_token()

    # Save creds to json file
    creds = {'CONSUMER_KEY':CONSUMER_KEY, 'CONSUMER_SECRET_KEY':CONSUMER_SECRET_KEY,
             'ACCESS_TOKEN':ACCESS_TOKEN, 'ACCESS_TOKEN_SECRET':ACCESS_TOKEN_SECRET}
    with open(TWITTER_CREDS_FN, 'w') as outfile:  
        json.dump(creds, outfile)

In [311]:
try:
    with open(TWITTER_CREDS_FN) as data_file:
        data = json.load(data_file)
        CONSUMER_KEY = data['CONSUMER_KEY']
        CONSUMER_SECRET_KEY = data['CONSUMER_SECRET_KEY']
        ACCESS_TOKEN = data['ACCESS_TOKEN']
        ACCESS_TOKEN_SECRET = data['ACCESS_TOKEN_SECRET']
#         print('CONSUMER_KEY={}\nCONSUMER_SECRET_KEY={}\nACCESS_TOKEN={}\nACCESS_TOKEN_SECRET={}'.format(
#             CONSUMER_KEY, CONSUMER_SECRET_KEY, ACCESS_TOKEN, ACCESS_TOKEN_SECRET))
except IOError:
    print("Could not read file: ~/.twitter-credentials.json. Please Instantiate object with your username and password")

In [312]:
# twitter = Twython(CONSUMER_KEY, access_token=ACCESS_TOKEN)
api = Twython(CONSUMER_KEY, access_token=ACCESS_TOKEN)


In [313]:
# total_tweets = []
# texts2=[]
# MAX_ATTEMPTS = 10000
# full_Text=''
# COUNT_OF_TWEETS_TO_BE_FETCHED = 448000



# for i in range(0,MAX_ATTEMPTS):

#     if(COUNT_OF_TWEETS_TO_BE_FETCHED < len(total_tweets)):
#         break # we got 500 tweets... !!
        
#     if(0 == i):
# #         results = twitter.search(q='virus',tweet_mode='extended', count=5000,geocode='43.131755,-75.489821,200km')['statuses']
# #         time.sleep(3)
#          results = twitter.search(q='virus',until='2020-04-9',tweet_mode='extended',result_type='mixed', count=5000,geocode='43.131755,-75.489821,200km')['statuses']
#     else:
#         # After the first call we should have max_id from result of previous call. Pass it in query.
# #         results = twitter.search(q='virus',tweet_mode='extended', count=5000,geocode='43.131755,-75.489821,200km')['statuses']
# #         time.sleep(3)
#          results = twitter.search(q='virus', until='2020-04-9',tweet_mode='extended',result_type='mixed', count=5000,geocode='43.131755,-75.489821,200km')['statuses']

#     # STEP 2: Save the returned tweets
#     count=0
#     for result in results: 
#         count=count+1
#         if 'retweeted_status' not in result :
#             pass
#         else:
#             if result['is_quote_status']==True :
#                 print("-----------------------------------------------------")
#                 pass
#             else:
#                 hashtags=[]
#                 for hashtag in result['entities']['hashtags']:
#                     hashtags.append(hashtag['text'])
#                 texts2 = [result['created_at'],result['full_text'],result['user']['created_at'],result['favorite_count'],result['retweet_count'],result['user']['location'],result['user']['followers_count'],result['user']['verified'],result['lang'],hashtags]
#                 total_tweets.append(texts2)
#                 full_Text= result['full_text']
#                 print('normal')
#                 print(full_Text)
    
# #     time.sleep(3)
                

In [314]:
# nyc= '41.799111,-74.146333,200km'
# chic='39.960612,-89.087748,300km'
# sa='36.967862,-121.743264,400km'
# la='33.597697,-117.782378,250km'
# fl='28.175202,-81.651903,300km'
# seoul='36.795872,127.912406,160km'
# ital='44.498498,11.104739,350km'
# uk='52.674922,-1.420274,300km'
# germany='53.131055,11.324677,300km'
# ind='23.529528,80.728972,700km'
# usa='40.614615,-113.692701,1000km'

In [322]:
total_tweets = []

def func():
    
    print('new start ---------')
    texts2=[]
    full_Text=''
    count=0
    MAX_ATTEMPTS = 100000
    COUNT_OF_TWEETS_TO_BE_FETCHED = 40000

    for i in range(0,MAX_ATTEMPTS):
        print(time.ctime(time.time()))
        print('OUTSIDE total_tweets = ',len(total_tweets))
        if (COUNT_OF_TWEETS_TO_BE_FETCHED < len(total_tweets)):
            print('total_tweets = ',len(total_tweets))
            print('count = ',count)
            print(time.ctime(time.time()))
            return total_tweets
            break

        # STEP 1: Query Twitter
        if(0 == i ):
                        
#             results = api.search(q='virus',tweet_mode='extended',count=5000)
#             results = api.search(q='economy',tweet_mode='extended',count=5000,geocode='28.175202,-81.651903,300km') 
            results = api.search(q="economy",count='500',until='2020-04-21',tweet_mode='extended',geocode='41.799111,-74.146333,200km')        
        else:
            # After the first call we should have max_id from result of previous call. Pass it in query.
            
#             results = api.search(q='virus',tweet_mode='extended',count=5000,include_entities='true',max_id=next_max_id)
#             results = api.search(q='economy',tweet_mode='extended',count=5000,geocode='28.175202,-81.651903,300km',include_entities='true',max_id=next_max_id)
            results = api.search(q="economy",count='4000',until='2020-04-21',tweet_mode='extended',geocode='41.799111,-74.146333,200km',include_entities='true',max_id=next_max_id)
            
            
    # STEP 2: Save the returned tweets

        for result in results['statuses']: 
            count=count+1
            if 'retweeted_status' in result:
                pass
            else:
                if result['is_quote_status']==True : 
                    pass
                else:
                    hashtags=[]
                    for hashtag in result['entities']['hashtags']:
                        hashtags.append(hashtag['text'])
                    texts2 = [result['created_at'],result['full_text'],result['user']['created_at'],result['favorite_count'],result['retweet_count'],result['user']['location'],result['user']['followers_count'],result['user']['verified'],result['lang'],hashtags]
                    total_tweets.append(texts2)
                    full_Text= result['full_text']

        try:
            # Parse the data returned to get max_id to be passed in consequent call.
            next_results_url_params = results['search_metadata']['next_results']
            next_max_id = next_results_url_params.split('max_id=')[1].split('&')[0]
            print(type(next_max_id))
            print('id  = ',next_max_id)
        except:
            print("oops")            
            print(time.ctime(time.time()))            
            break

In [323]:
func()

new start ---------
Wed Apr 22 17:00:28 2020
OUTSIDE total_tweets =  0
<class 'str'>
id  =  1252385200773885954
Wed Apr 22 17:00:29 2020
OUTSIDE total_tweets =  19
<class 'str'>
id  =  1252383800471875590
Wed Apr 22 17:00:29 2020
OUTSIDE total_tweets =  34
<class 'str'>
id  =  1252382748628533247
Wed Apr 22 17:00:29 2020
OUTSIDE total_tweets =  54
<class 'str'>
id  =  1252381602631450624
Wed Apr 22 17:00:30 2020
OUTSIDE total_tweets =  72
<class 'str'>
id  =  1252380272198205442
Wed Apr 22 17:00:30 2020
OUTSIDE total_tweets =  84
<class 'str'>
id  =  1252379193691664386
Wed Apr 22 17:00:30 2020
OUTSIDE total_tweets =  107
<class 'str'>
id  =  1252377990362656767
Wed Apr 22 17:00:31 2020
OUTSIDE total_tweets =  121
<class 'str'>
id  =  1252376958639288321
Wed Apr 22 17:00:31 2020
OUTSIDE total_tweets =  133
<class 'str'>
id  =  1252375962580205572
Wed Apr 22 17:00:32 2020
OUTSIDE total_tweets =  147
<class 'str'>
id  =  1252375066001592322
Wed Apr 22 17:00:32 2020
OUTSIDE total_tweets =

<class 'str'>
id  =  1252285958990565375
Wed Apr 22 17:00:54 2020
OUTSIDE total_tweets =  1585
<class 'str'>
id  =  1252284578385072127
Wed Apr 22 17:00:55 2020
OUTSIDE total_tweets =  1611
<class 'str'>
id  =  1252282930849886207
Wed Apr 22 17:00:55 2020
OUTSIDE total_tweets =  1631
<class 'str'>
id  =  1252281521173561344
Wed Apr 22 17:00:56 2020
OUTSIDE total_tweets =  1656
<class 'str'>
id  =  1252280017460969478
Wed Apr 22 17:00:56 2020
OUTSIDE total_tweets =  1678
<class 'str'>
id  =  1252278794821353483
Wed Apr 22 17:00:56 2020
OUTSIDE total_tweets =  1701
<class 'str'>
id  =  1252277081003700223
Wed Apr 22 17:00:57 2020
OUTSIDE total_tweets =  1726
<class 'str'>
id  =  1252275832825024513
Wed Apr 22 17:00:57 2020
OUTSIDE total_tweets =  1745
<class 'str'>
id  =  1252274418115641346
Wed Apr 22 17:00:58 2020
OUTSIDE total_tweets =  1765
<class 'str'>
id  =  1252273127628955647
Wed Apr 22 17:00:58 2020
OUTSIDE total_tweets =  1782
<class 'str'>
id  =  1252271533386596354
Wed Apr 2

<class 'str'>
id  =  1252075508818489344
Wed Apr 22 17:01:21 2020
OUTSIDE total_tweets =  3168
<class 'str'>
id  =  1252073599709650945
Wed Apr 22 17:01:21 2020
OUTSIDE total_tweets =  3176
<class 'str'>
id  =  1252072201383227393
Wed Apr 22 17:01:21 2020
OUTSIDE total_tweets =  3182
<class 'str'>
id  =  1252070601952497664
Wed Apr 22 17:01:21 2020
OUTSIDE total_tweets =  3193
<class 'str'>
id  =  1252068915800207359
Wed Apr 22 17:01:22 2020
OUTSIDE total_tweets =  3203
<class 'str'>
id  =  1252067390973603842
Wed Apr 22 17:01:22 2020
OUTSIDE total_tweets =  3221
<class 'str'>
id  =  1252065736954183679
Wed Apr 22 17:01:22 2020
OUTSIDE total_tweets =  3235
<class 'str'>
id  =  1252063851140374527
Wed Apr 22 17:01:23 2020
OUTSIDE total_tweets =  3246
<class 'str'>
id  =  1252062458954444799
Wed Apr 22 17:01:23 2020
OUTSIDE total_tweets =  3252
<class 'str'>
id  =  1252060919733125120
Wed Apr 22 17:01:23 2020
OUTSIDE total_tweets =  3258
<class 'str'>
id  =  1252059306805035014
Wed Apr 2

<class 'str'>
id  =  1251974752144633857
Wed Apr 22 17:01:45 2020
OUTSIDE total_tweets =  3997
<class 'str'>
id  =  1251973333236637695
Wed Apr 22 17:01:45 2020
OUTSIDE total_tweets =  4010
<class 'str'>
id  =  1251972225630076929
Wed Apr 22 17:01:46 2020
OUTSIDE total_tweets =  4021
<class 'str'>
id  =  1251970957666848770
Wed Apr 22 17:01:46 2020
OUTSIDE total_tweets =  4029
<class 'str'>
id  =  1251969962844909569
Wed Apr 22 17:01:46 2020
OUTSIDE total_tweets =  4039
<class 'str'>
id  =  1251968928269639681
Wed Apr 22 17:01:46 2020
OUTSIDE total_tweets =  4048
<class 'str'>
id  =  1251967969019793407
Wed Apr 22 17:01:47 2020
OUTSIDE total_tweets =  4059
<class 'str'>
id  =  1251966888554438664
Wed Apr 22 17:01:47 2020
OUTSIDE total_tweets =  4072
<class 'str'>
id  =  1251965962787618815
Wed Apr 22 17:01:47 2020
OUTSIDE total_tweets =  4080
<class 'str'>
id  =  1251965159431643136
Wed Apr 22 17:01:47 2020
OUTSIDE total_tweets =  4087
<class 'str'>
id  =  1251964372211036159
Wed Apr 2

<class 'str'>
id  =  1251895355161161734
Wed Apr 22 17:02:08 2020
OUTSIDE total_tweets =  4895
<class 'str'>
id  =  1251893620883574784
Wed Apr 22 17:02:08 2020
OUTSIDE total_tweets =  4916
<class 'str'>
id  =  1251892279138308100
Wed Apr 22 17:02:08 2020
OUTSIDE total_tweets =  4933
<class 'str'>
id  =  1251890841590861824
Wed Apr 22 17:02:09 2020
OUTSIDE total_tweets =  4946
<class 'str'>
id  =  1251889720621256703
Wed Apr 22 17:02:09 2020
OUTSIDE total_tweets =  4961
<class 'str'>
id  =  1251888393165905928
Wed Apr 22 17:02:09 2020
OUTSIDE total_tweets =  4980
<class 'str'>
id  =  1251886989068120063
Wed Apr 22 17:02:09 2020
OUTSIDE total_tweets =  5002
<class 'str'>
id  =  1251885600258523135
Wed Apr 22 17:02:10 2020
OUTSIDE total_tweets =  5021
<class 'str'>
id  =  1251884391539904512
Wed Apr 22 17:02:10 2020
OUTSIDE total_tweets =  5035
<class 'str'>
id  =  1251883422458564608
Wed Apr 22 17:02:10 2020
OUTSIDE total_tweets =  5054
<class 'str'>
id  =  1251882367989567493
Wed Apr 2

<class 'str'>
id  =  1251694298271285248
Wed Apr 22 17:02:31 2020
OUTSIDE total_tweets =  5990
<class 'str'>
id  =  1251693082212364287
Wed Apr 22 17:02:31 2020
OUTSIDE total_tweets =  6000
<class 'str'>
id  =  1251691881903333375
Wed Apr 22 17:02:31 2020
OUTSIDE total_tweets =  6009
<class 'str'>
id  =  1251690446969081858
Wed Apr 22 17:02:31 2020
OUTSIDE total_tweets =  6018
<class 'str'>
id  =  1251689163864313856
Wed Apr 22 17:02:32 2020
OUTSIDE total_tweets =  6026
<class 'str'>
id  =  1251687858617884672
Wed Apr 22 17:02:32 2020
OUTSIDE total_tweets =  6039
<class 'str'>
id  =  1251686760335257600
Wed Apr 22 17:02:32 2020
OUTSIDE total_tweets =  6051
<class 'str'>
id  =  1251685484046290944
Wed Apr 22 17:02:32 2020
OUTSIDE total_tweets =  6062
<class 'str'>
id  =  1251684195845488642
Wed Apr 22 17:02:33 2020
OUTSIDE total_tweets =  6076
<class 'str'>
id  =  1251682865609310207
Wed Apr 22 17:02:33 2020
OUTSIDE total_tweets =  6081
<class 'str'>
id  =  1251681701115260929
Wed Apr 2

TwythonRateLimitError: Twitter API returned a 429 (Too Many Requests), Rate limit exceeded

In [324]:
len(total_tweets)

6124

In [325]:
df_US=pd.DataFrame(total_tweets,columns=['date','tweet','created','fav','number of retweets','location','follower','verified','language','hashtags'])
# df_US.language.describe()
# df_US[df_US.language=="en"]
df_US

,date,tweet,created,fav,number of retweets,location,follower,verified,language,hashtags
0,Mon Apr 20 23:59:46 +0000 2020,The governor of the state where my family and ...,Sat Feb 02 22:30:16 +0000 2019,0,0,"Manhattan, NY",12,False,en,[]
1,Mon Apr 20 23:59:37 +0000 2020,@adamjohnsonNYC @burnlittlelight Never mind wh...,Thu Jul 21 23:27:07 +0000 2016,0,0,"New York, NY",171,False,en,[]
2,Mon Apr 20 23:59:10 +0000 2020,why do you only care about the southern portio...,Sat Oct 13 22:13:36 +0000 2018,0,0,"Maine, USA",20,False,en,[]
3,Mon Apr 20 23:59:07 +0000 2020,@TerryGalvanChi In this economy though? The De...,Sun Apr 10 05:04:03 +0000 2011,1,0,"New York, NY",41,False,en,[]
4,Mon Apr 20 23:59:05 +0000 2020,"Effect...”So many things are broken, so many p...",Sun Feb 21 23:11:24 +0000 2010,0,0,Connecticut,182,False,en,[COVID19]
...,...,...,...,...,...,...,...,...,...,...
6119,Sun Apr 19 01:00:49 +0000 2020,It’s over in more ways. Voting ⁦@realDonaldTru...,Mon May 23 17:01:58 +0000 2011,0,0,"Florida, NY",264,False,en,[]
6120,Sun Apr 19 01:00:48 +0000 2020,@joelockhart @realDonaldTrump PS— a #tradedeal...,Thu Feb 21 23:29:59 +0000 2013,0,0,"Garden City, NY",292,False,en,"[tradedeal, American, Trump, US]"
6121,Sun Apr 19 01:00:32 +0000 2020,Florida’s economy is collapsing under COVID-19...,Mon Aug 25 21:03:10 +0000 2008,27,14,Saugerties NY,13242,False,en,[]
6122,Sun Apr 19 01:00:17 +0000 2020,@kimKBaltimore @GovLarryHogan The virus is som...,Sat Jul 19 15:03:23 +0000 2014,0,0,"Maryland, USA",7043,False,en,[]


In [326]:
from nltk.corpus import stopwords
import nltk 
nltk.download('stopwords')

stop_words1 = set(stopwords.words('english')) | set(['would', '#', '@'])
# df_title = pd.read_csv('data/titles_65k.csv', index_col=0)
df_US['tweet'] = df_US['tweet'].apply(lambda title: ' '.join([word for word in title.split() if word not in stop_words1]))
# df_title.to_csv('data/titles.csv')
df_US['tidy_tweet'] = df_US['tweet'].str.replace("[^a-zA-Z#]", " ")
df_US.head(10)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Che10\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,date,tweet,created,fav,number of retweets,location,follower,verified,language,hashtags,tidy_tweet
0,Mon Apr 20 23:59:46 +0000 2020,The governor state family friends live decided...,Sat Feb 02 22:30:16 +0000 2019,0,0,"Manhattan, NY",12,False,en,[],The governor state family friends live decided...
1,Mon Apr 20 23:59:37 +0000 2020,@adamjohnsonNYC @burnlittlelight Never mind ag...,Thu Jul 21 23:27:07 +0000 2016,0,0,"New York, NY",171,False,en,[],adamjohnsonNYC burnlittlelight Never mind ag...
2,Mon Apr 20 23:59:10 +0000 2020,care southern portion state? 2 cases County 2 ...,Sat Oct 13 22:13:36 +0000 2018,0,0,"Maine, USA",20,False,en,[],care southern portion state cases County ...
3,Mon Apr 20 23:59:07 +0000 2020,@TerryGalvanChi In economy though? The Devil p...,Sun Apr 10 05:04:03 +0000 2011,1,0,"New York, NY",41,False,en,[],TerryGalvanChi In economy though The Devil p...
4,Mon Apr 20 23:59:05 +0000 2020,"Effect...”So many things broken, many prices g...",Sun Feb 21 23:11:24 +0000 2010,0,0,Connecticut,182,False,en,[COVID19],Effect So many things broken many prices g...
5,Mon Apr 20 23:58:37 +0000 2020,4) Meanwhile 6-8 residents become severely ill...,Sun Nov 27 04:09:11 +0000 2011,1,0,"Georgia, USA",17,False,en,[],Meanwhile residents become severely ill...
6,Mon Apr 20 23:57:46 +0000 2020,@GovMurphy No. The hospitals weren’t even clos...,Tue Nov 08 22:36:35 +0000 2016,3,0,"High Bridge, NJ",28,False,en,[],GovMurphy No The hospitals weren t even clos...
7,Mon Apr 20 23:57:00 +0000 2020,@AGWilliamTong Time open economy,Mon Mar 14 19:34:55 +0000 2011,0,0,"Greenwich, CT, USA",47,False,en,[],AGWilliamTong Time open economy
8,Mon Apr 20 23:56:49 +0000 2020,Gov. Ned Lamont painted picture Monday state e...,Wed Mar 04 18:57:23 +0000 2009,1,1,Connecticut,2046,False,en,[],Gov Ned Lamont painted picture Monday state e...
9,Mon Apr 20 23:56:22 +0000 2020,"@AliHamza78910 Man, I’d love (as consumer, ign...",Wed Apr 09 03:50:19 +0000 2008,1,1,"Central Islip, NY",190,False,en,[],AliHamza Man I d love as consumer ign...


In [329]:
# df_US.to_csv(r'ny_21_updated_tweet_eco.csv', index = False)